In [1]:
from scripts import support, insd, kuroda, nras, metrics, vectorization as v
import pandas as pd
import numpy as np

In [2]:
# Load base matrix
df = pd.read_excel("./data/all2011.xlsx", 'ТИоц',skiprows=1,index_col=2)
df = df.iloc[2:-11,2:-16]
df = df.astype(float)
# Load predicted matrix
pred_df = pd.read_excel("./data/tri-2012.xlsx", 'ТИоц',skiprows=1,index_col=2)
pred_df = pred_df.iloc[2:-12,2:-11]
pred_df = pred_df.astype(float)
shape = pred_df.shape


In [3]:
# Sums of rows\columns
sums_rows = pred_df.sum(axis=1).values.reshape(-1,1)
sums_cols = pred_df.sum(axis=0).values.reshape(-1,1)

In [4]:
# Vectorization of matrix
vect_a = v.tovector(df.values) #df.unstack().values.reshape(-1,1)
vect_a

array([[7.02865e+05],
       [8.83000e+02],
       [6.40000e+01],
       ...,
       [0.00000e+00],
       [0.00000e+00],
       [0.00000e+00]])

In [5]:
# Add coefficients of constraints & merge column of constraints
c = np.append(sums_rows,sums_cols,axis=0)
G = v.addrows(shape)
G = v.addcolumns(shape,G)
#G = v.addrows(shape,G,sparsed=True) # sparsed
G

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [6]:
# Run INSD
res_df = insd.insd(aa=vect_a,c=c, G=G)
metrics.get_values(res_df,pred_df.values)
pd.DataFrame(v.tomatrix(res_df)).round(3).head()

Academic license - for non-commercial use only
Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
N0 390
MAPE 12.0875
SWAD 0.0256
WAPE 6.0546


,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,703254.998,710.988,446.138,3.075,189.392,0.000,5.789,116.212,1397528.792,36145.869,...,1273.473,765.666,37011.175,4813.264,21093.756,132.718,679.351,1462.063,3678.361,0.0
1,908.877,17381.577,1.120,143.331,132.954,0.000,21.358,17.004,358.219,0.000,...,17.980,68.644,6297.640,158.427,257.520,14.074,19.805,158.741,135.816,0.0
2,63.981,0.000,5056.309,0.000,0.000,0.000,1.157,0.000,77703.387,0.000,...,84.249,11.853,705.885,12.022,363.301,1.143,34.262,107.114,0.000,0.0
3,1375.485,8.417,47.945,80800.005,410.626,0.707,360.469,146.950,1623.731,0.000,...,38.170,209.673,2761.132,561.983,2384.373,140.818,19.478,185.625,216.505,0.0
4,252.023,0.000,0.000,356.735,320588.748,0.000,0.000,229.470,105.698,0.000,...,7.699,4809.419,0.000,0.000,13.573,2.665,0.000,0.000,0.000,0.0


In [7]:
# Run NRAS
res_df = nras.nras(G=G,aa=vect_a,c=c,accuracy=0.01, limit =  100)
metrics.get_values(res_df,pred_df.values)

23
N0 390
MAPE 11.9702
SWAD 0.0249
WAPE 6.0148


In [8]:
# Run Kuroda
res_df = kuroda.kuroda(G=G,aa=vect_a,c=c,mtype=1)
metrics.get_values(res_df,pred_df.values)

Parameter BarConvTol unchanged
   Value: 1e-08  Min: 0.0  Max: 1.0  Default: 1e-08
Parameter BarQCPConvTol unchanged
   Value: 1e-06  Min: 0.0  Max: 1.0  Default: 1e-06
Changed value of parameter DualReductions to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
N0 12
MAPE 12.1578
SWAD 0.0313
WAPE 6.3198


### Загрузка\сохранение данных

In [48]:
# Загружаем данные из одной несимметричной таблицы 
quadrant1, df, codes_industries, codes_products = support.load_Rosstat_non_sym("./data/tri-2012.xlsx", 'М-имп', quandrant2_columns = 11, quadrant3_rows = 2)

Обрабатываем данные из таблицы в формате Rosstat "ТАБЛИЦА ИСПОЛЬЗОВАНИЯ ИМПОРТНОЙ ПРОДУКЦИИ за 2012 год (млн. рублей)"


In [ ]:
# Сохраняем таблицы в файл постранично 
df_d[0].columns.name = "Отечественный выпуск0"
result_tables = {support.pack_name("Отечественный выпуск0",years,'2008'): df_d[0]}
support.save_to_excel("test_res.xlsx", years, codes, **result_tables)

In [44]:
support.pack_name("мяумя",'2020-21000','2019')

'мяумя за 2020-21000гг(2019)'